In [ ]:
'''

整體功能描述

'''

'\n\n整體功能描述\n\n'

In [53]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))  # 載入line_secret_key資訊
server_url=secretFileContentJson.get("server_url")  # 讀取webhooks網址

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/python/factory/test/YY_new/line/YY",   # url連結名稱(https://ngrok/static_url_path/)
            static_folder = "/python/factory/test/YY_new/line/YY")  #  https://ngrok/static_url_path/對應到的local資料夾位置
# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))  # 讀取line channel_access_token
handler = WebhookHandler(secretFileContentJson.get("secret_key"))  # 讀取line secret_key

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [54]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *
import sys

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName,encoding="utf8") as f:  # 在windows環境下執行須進行編碼才可讀取中文字 [encoding="utf8"]
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換，判斷讀取的json檔是哪一種line格式
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))    

    # 回傳
    return returnArray

In [55]:
# 呼叫樹莓派執行拍照並回傳資料的動作
import paramiko

def par(ids):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("IP",22,"username", "password")  # 網路連線設定
    stdin, stdout, stderr = ssh.exec_command("python3 /home/pi/Desktop/test.py %s" % ids)  # 呼叫執行檔並傳送參數
    paramiko_result = stdout.readlines()  # 回傳產出結果
    print(paramiko_result)
    ssh.close()
    return paramiko_result

In [56]:
import cv2
import numpy as np

def get_photo_from_pi(ids):
    # 建立資料庫連線
    db = pymysql.connect(host='127.0.0.1',
                         port=3306,
                         user='yy_jimmy',
                         password='yyjimmy',
                         db='yyostech')
    # 使用 cursor() 建立 cursor 物件
    cursor = db.cursor()
    # 使用 execute()  執行 SQL 查詢
    cursor.execute("SELECT * from raspi_data where photo_id = \'%s\'" % ids)
    # 使用 fetchone() 取出一筆資料
    data = cursor.fetchone()
    # 關閉資料庫連線
    db.close()

    frame = data[1]
    # converting bytestring frame into imshow argument
    nparr = np.fromstring(frame, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    newimg = cv2.resize(frame,(800,600))
    # 寫入圖檔
    cv2.imwrite('./YY/test.jpg', newimg)
    return 'image save!'

In [57]:
import pymysql
# 儲存使用者資料
def user_follow(line_id, user_name, right_id):
    # 連線資料庫
    db = pymysql.connect(host='127.0.0.1',
                         port=3306,
                         user='yy_jimmy',
                         password='yyjimmy',
                         db='yyostech')
    # 建立連線物件
    cursor = db.cursor()
    sql_insert_blob_query = """select `line_id` from user_profile where `line_id` = %s"""
    insert_blob_tuple1 = (line_id, )
    cursor.execute(sql_insert_blob_query, insert_blob_tuple1)
    data = cursor.fetchone()
    if data == None:
        # user_profile
        sql_insert_blob_query = """INSERT INTO user_profile (`line_id`, `user_name`, `right_id`) VALUES (%s, %s, %s)"""
        insert_blob_tuple2 = (line_id, user_name, right_id)
        cursor.execute(sql_insert_blob_query, insert_blob_tuple2)
        # 確認執行上述指令
        result = db.commit()
        # 關閉資料庫連線
        db.close()
    else:
        result = 'user exist'
    return result


def get_menu_id(line_id):
    # 連線資料庫
    db = pymysql.connect(host='127.0.0.1',
                         port=3306,
                         user='yy_jimmy',
                         password='yyjimmy',
                         db='yyostech')
    # 建立連線物件
    cursor = db.cursor()
    # user_profile
    sql_insert_blob_query = """select b.menu_id from user_profile a inner join user_right b on a.right_id = b.right_id and a.line_id = %s"""
    insert_blob_tuple = (line_id, )
    cursor.execute(sql_insert_blob_query, insert_blob_tuple)
    data = cursor.fetchone()
    # 關閉資料庫連線
    db.close()
    return data[0]


In [58]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests

# 消息清單
reply_message_list = [
    TextSendMessage(text="您好! 歡迎使用研耘發芽率辨識操作的頻道")
]

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    line_id = eval(str(user_profile))['userId']
    user_name = eval(str(user_profile))['displayName']
    right_id = 0  # 測試權限  
    print(line_id, user_name, right_id)
    user_follow(line_id, user_name, right_id)
    
    # 將菜單綁定在用戶身上
    linkRichMenuId = get_menu_id(line_id)
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)  # 指定user_id檔定rich_menu_id
    linkMenuRequestHeader={'Content-Type':'image/png','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}  # request header資訊(含channel_access_token)
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)  # 致行榜定圖文選單的命令
    
    # 回覆文字消息與圖片消息(綁定圖文選單與回文是不同的指令喔~~)
    line_bot_api.reply_message(
        event.reply_token,
        reply_message_list
    )

In [59]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''
import time
# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    if gerData['saw_date'] != None and gerData['done'] == 0:  # 3收到種子批號，回覆：請確認資料
        if len(event.message.text) == 10:  # 確認種子批號的資料格式
            gerData['seed_numbers'] = event.message.text  # 種子批號
            gerData['done'] = 1  # 紀錄完成
            
            result_message_array =[]
            replyJsonPath = "./YY/ger4.json"
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
        else:  # 批號資料錯誤重新輸入
            result_message_array =[]
            replyJsonPath = './YY/ger3.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
        print(gerData)
    else:
        print(event.message.text)

In [60]:
gerData = {'process_id': None, 'series_id':None, 'seed_numbers':None, 'identify_date':None, 'saw_date': None, 'done': None, 'threshold': None, 'germination_percent': None}  # 作為執行發芽率辨識的暫存物件
humanData = {'name':None, 'date': None, 'image':None, 'done':0}  # 作為執行人工判別的暫存物件0-10
# addData
# tableData
# powerData
# problemData

In [61]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 
from datetime import datetime

@handler.add(PostbackEvent)
def process_postback_event(event):
    global gerData
    dict1 = parse_qs(event.postback.data)
    if 'start' == list(dict1.keys())[0]:  # 0圖文選單，第一層動作判斷(start)
        func = list(dict1.values())[0][0]
        if func == 'ger1' or func == 'human1':  # 辨識發芽率跟人工判別
            print('get data from series_data')
            result_message_array =[]
            replyJsonPath = './YY/' + func + '.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 讀資料庫改json
            # ger: 品種名稱、id、縮圖，點more要再列出其他品種
            # human: 品種名稱、id、縮圖，點more要再列出其他品種
            # table: 品種名稱、id、縮圖，點more要再列出其他品種
            # power: 列出現有人員清單
            # problem、add: 文字方框而已，用dict紀錄啟動該功能
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        
    if 'ger' in list(dict1.keys())[0]:  # 發芽率辨識
        print(dict1.values())
        dict2 = parse_qs(list(dict1.values())[0][0])
        print(dict2)
        
        if 'getdate' in list(dict2.keys())[0]:  # 1選擇了作物編號，回覆：請輸入播種日期
            gerData['series_id'] = list(dict2.values())[0][0]  # 作物編號
            gerData['identify_date'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 程序開始時間
            print(gerData)
            
            result_message_array =[]
            replyJsonPath = './YY/ger2.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
        if 'selectDate' in list(dict2.keys())[0]:  # 2收到播種日期，回覆：請輸入種子批號
            gerData['saw_date'] = event.postback.params['date']  # 播種日期
            gerData['done'] = 0  # 紀錄可以進入下一階段
            print(gerData)
            
            result_message_array =[]
            replyJsonPath = './YY/ger3.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
        if 'confirm' in list(dict2.keys())[0]:  # 4確認種子批號，回覆：請就位準備拍照
            print('get data from series_data')  # 從資料庫抓辨識用的參數
            gerData['threshold'] = 0
            gerData['germination_percent'] = 0
            
            result_message_array =[]
            replyJsonPath = './YY/ger5.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
        if 'retype' in list(dict2.keys())[0]:  # 4a資料有錯重來
            gerData = {'process_id': None, 'series_id':None, 'seed_numbers':None, 'identify_date':None, 'saw_date': None, 'done': None, 'threshold': None, 'germination_percent': None}  # 作為執行發芽率辨識的暫存物件

        if 'shot' in list(dict2.keys())[0]:  # 5開始拍照，回覆：請確認拍照成像
            photo_id = str(datetime.now().strftime("%Y%m%d%H%M%S")) # 產生raspi_data照片編號
#             result = par(photo_id)  # 呼叫樹莓派拍照
            result = 'ok'
            print(result, photo_id)
            
            if result == 'ok':  # 回傳OK再去資料庫抓資料
#                 image, image_show = get_photo_from_pi(gerData['id'])  # 抓剛剛存到資料庫的照片(顯示(image_show)跟使用(image)的分開存)
                result_message_array =[]
                replyJsonPath = './YY/ger6.json'
                result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
                line_bot_api.reply_message(event.reply_token, result_message_array)
                
        if 'check' in list(dict2.keys())[0]:  # 6確認成像正常，回覆：辨識結果簡述
            print('模型辨識回傳結果、回存資料庫')
            
            result_message_array =[]
            replyJsonPath = './YY/ger7.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
        if 'stop' in list(dict2.keys())[0]:  # 7確認停止辨識，回覆：本次流程辨識結果一覽
            # 抓取這一批process_id的所有辨識紀錄
            print('get this process_id\'s data')
            # reset dict
            gerData = {'process_id': None, 'series_id':None, 'seed_numbers':None, 'identify_date':None, 'saw_date': None, 'done': None, 'threshold': None, 'germination_percent': None}  # 作為執行發芽率辨識的暫存物件

            result_message_array =[]
            replyJsonPath = './YY/ger8.json'
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
            line_bot_api.reply_message(event.reply_token, result_message_array)
            
    if 'human' in list(dict1.keys())[0]:  # 人工判別
        print(dict1.values())
        dict2 = parse_qs(list(dict1.values())[0][0])
        print(dict2)
        if 'species' in list(dict2.keys())[0]:  # 確認判別的品種
            humanData['name'] = list(dict2.values())[0][0]
            humanData['date'] = '2020/3/7'
            # 抓所有待判斷照片出來
            # 回傳一張照片的URL
            replyJsonPath = './YY/human2.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
            
        if 'check' in list(dict2.keys())[0] and humanData['done'] == 9:  # 判斷是否停止
            print(list(dict2.values())[0][0])
            print(humanData['done'], 'done')
            replyJsonPath = './YY/human4.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
            
        if 'check' in list(dict2.keys())[0] and humanData['done'] < 9:  # 確認判斷的數量
            humanData['done'] = humanData['done'] + 1
            print(humanData['done'])
            print(list(dict2.values())[0][0])
            replyJsonPath = './YY/human2.json'  # 回應檔路徑
            result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
            # 回傳整體辨識結果(文字)
            line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
        
        if 'continue' in list(dict2.keys())[0]:
            if 'yes' in list(dict2.values())[0][0]:  # 繼續在辨識十張照片
                humanData['done'] = 0
                replyJsonPath = './YY/human2.json'  # 回應檔路徑
                result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
                # 回傳整體辨識結果(文字)
                line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應
                
            if 'no' in list(dict2.values())[0][0]:  # 停止辨識並儲存
                humanData['done'] = 0
                replyJsonPath = './YY/human5.json'  # 回應檔路徑
                result_message_array = detect_json_array_to_new_message_array(replyJsonPath)  # 呼叫回應檔
                # 回傳整體辨識結果(文字)
                line_bot_api.reply_message(event.reply_token, result_message_array)  # post回應

In [62]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


get data from series_data


127.0.0.1 - - [23/Mar/2020 23:49:31] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2020 23:49:32] "POST / HTTP/1.1" 200 -


辨識發芽率
芝麻葉(3G1)
dict_values([['getdate=3g1']])
{'getdate': ['3g1']}
{'process_id': None, 'series_id': '3g1', 'seed_numbers': None, 'identify_date': '2020-03-23 23:49:32', 'saw_date': None, 'done': None, 'threshold': None, 'germination_percent': None}


...

重拍


127.0.0.1 - - [24/Mar/2020 00:05:25] "GET /python/factory/test/YY_new/line/YY/3G1_rect1.jpg HTTP/1.1" 200 -


dict_values([['shot=no']])
{'shot': ['no']}
ok 20200324000549


127.0.0.1 - - [24/Mar/2020 00:05:49] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2020 00:05:50] "GET /python/factory/test/YY_new/line/YY/test.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Mar/2020 00:05:50] "POST / HTTP/1.1" 200 -


重拍


In [63]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])

'\n\nApplication 運行（heroku版）\n\n'

# 工作進度：
* 2020/3/2
    - 完成後台骨架與json模板
    - 完成發芽率辨識1-3
    
* 2020/3/5
    - 修改發芽率辨識模組(都用postback接收指令)
    - 整理if else架構
    
* 問題
    - 線下圖片檔(圖檔或代碼型態)如何讀取(已解決圖檔問題，代碼格式尚未測試)
    - 每個動作紀錄於暫存檔時需紀錄操作者id，不可混淆不同人的操作歷程
    
    
    